In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm,tree
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer,scale,LabelEncoder, OneHotEncoder
from gensim.models import Word2Vec
from sklearn.metrics import r2_score,mean_squared_error, explained_variance_score, accuracy_score
from scipy import sparse
from scipy.sparse import hstack,vstack
import math
# Deep Learning 
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout, Activation
from keras.optimizers import SGD
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline

C:\Users\Dell\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\Dell\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
df = pd.read_csv('Claims2.csv',encoding='mac_roman', low_memory=False)

# Idekw2d

In [3]:
def plot_corr(df,size=20):
    corr = df.corr()
    fig, ax = plt.subplots(figsize=(size, size))
    b=ax.matshow(corr)
    fig.colorbar(b)
    ax.grid(which='minor',color='w', linestyle='-', linewidth=2)
    plt.xticks(range(len(corr.columns)), corr.columns)
    plt.yticks(range(len(corr.columns)), corr.columns)
    plt.show()
    a=sns.heatmap(corr, xticklabels=corr.columns.values,yticklabels=corr.columns.values, linewidth=0.5)

In [4]:
df.isnull().sum()

OCCURRENCE       0
LOSSDATE         0
LOB              0
COVERAGE         0
STATE            0
DESCR           26
CADDR1        2838
CSTATE         249
CZIP          4948
LOCNM          185
IZIP            67
DRIVERAGE        0
INCURRED         0
PREMIUM       7863
TOTALPAID        0
dtype: int64

# idekw2d - part 2

In [5]:
df1 = pd.read_csv('Claims.csv',encoding='mac_roman', low_memory=False)
columns_to_be_dropped =['PLATE','PLATESTATE','EMPLSTATUS','EMPLSTATUSDESC','EMPLSTATUS_NCCI','DOB','DATEOFHIRE','NCCI_NOI','NCCI_SOI','NCCI_SEV','JOBCODE','JOBTITLE','CLASSCODE','CLASSCODE_DESCR','IPHONE','IWORKPHONE','LNAME','LTAXID','LADDR1','LADDR2','LCITY','LSTATE','LZIP','LPHONE','LFAX','DNAME','DTAXID','DADDR1','DADDR2','DCITY','DSTATE','DZIP','DPHONE','DFAX','CLASSCODE','CLASSCODE_DESCR','DATEINITIA','DATERETURN','DATEREPORT','WEEKLYWAGE','DIVTXTCODE','INSUIT','DEFENSECOUNSEL','SUITRECEIVED','APPARENTSERVICEDATE','SUITASSIGNED','STIPDATE','DCTAXID','DCCONTACT','DCADDR1','DCADDR2','DCCITY','DCSTATE','DCZIP','DCPHONE','DCFAX','ISSNUM',
'CLIENTSCLMNO','THIRDNUM','MARITAL','INDRESERVES','MEDRESERVES','EXPRESERVES','TOTALRESERVES','PHP_1st_Deducible_Notice','PHP_AMSLocs','PHP_Baileys_Custom_Location','PHP_Corrigan_Location','PHP_Daryl_Flood_Locs','PHP_Date_of_Expected_Denial','PHP_Date_of_Recovery','PHP_Deductible_Explanation','PHP_Deductible_Verifies?','PHP_Employee_Theft','PHP_Hazen_CC#','PHP_Mysterious_Disappearance','NET_INDEMNITY_RESERVE','NET_EXPENSE_RESERVE','PHP_Order_Number','PHP_Owner_Retained_Salvage_Amount','PHP_Cause_Code','PHP_Ref._to_Carrier_for_Recovery','PHP_Salvage','PHP_TF_Final_Mile_Branch_Code','PHP_TF/Final_Mile_Contractor','PHP_UST_Branch_Code','PHP_Van_Line','PHP_Voluntary_Payment']
df1.drop(columns=columns_to_be_dropped, inplace=True)
df1.isnull().sum()

CLMNO               0
OCCURRENCE          0
CLNTNO              0
CARRIERCLMNO     6546
ADJUSTER           91
LOSSDATE            0
STATUS              0
OPENDATE            0
CLOSEDATE         111
REOPENDATE      18460
LOB                 0
COVERAGE            0
STATE               0
DESCR              26
DESCR2          18018
INJURIES        13376
CAUSE           13861
ANATOMY             0
IPOLNO              0
IEFFDT              0
IEXPDT              0
CLAIMANT            0
CSSNUM              0
CADDR1           2838
CADDR2          19213
CCITY            4188
CSTATE            249
CZIP             4948
CPHONE          11426
CWORKPHONE      17490
CGENDER           318
LOCDESC          1430
LOC                 0
LOCTXTCODE        185
LOCNM             185
LOCNM2            201
LOCCITY           214
LOCSTATE          214
LOCZIP            252
DIVNUM              0
INAME               0
IADDR1             16
IADDR2          19909
ICITY              29
ISTATE              0
IZIP      

In [6]:
df1.iloc[:,14]

0                                     NaN
1                                     NaN
2                                     NaN
3                                     NaN
4                                  DAMAGE
5                                     NaN
6                                     NaN
7                                     NaN
8         THAT PIERCED A PIPE IN THE WALL
9                                     NaN
10                                    NaN
11                                    NaN
12                                    NaN
13                                    NaN
14                                    NaN
15                                    NaN
16                                    NaN
17                                    NaN
18                                    NaN
19                                    NaN
20                                    NaN
21                                NG MOVE
22                                    NaN
23                                

In [7]:
df1['DRIVERAGE'].dtype

dtype('O')

In [8]:
def label(df):
    #print(df.isnull().sum())
    df.fillna(0)
    X = df.iloc[:,0:len(df.iloc[0])].values
    for i in range(len(df.iloc[0])):
       # print(i, end='  ')
        labelencoder_X_1 = LabelEncoder()
        X[:, i] = labelencoder_X_1.fit_transform(X[:, i])
        #print(i)
    X=X.astype(np.float)
    d1=pd.DataFrame(X)
    return d1

In [9]:
def fillN(df):
    l=len(df.iloc[0])-1
    for i in range(l):
        #print(i)
        if(df1.isnull().sum()[i]>0):
            t=df.dtypes[i]
            for j in range(len(df)):
                value=df.iloc[j,i]
                
                if(t=='O'):
                    #if(i==14):
                     #   print(value, end=' ')
                    if(type(value)==str):
                        if(value.strip()==''):
                            df.iloc[j,i]='Unknown'
                    elif(math.isnan(value)):
                            df.iloc[j,i]='Unknown'
                    #if(i==14) : print(df.iloc[j,i])
                else:
                    if(math.isnan(value)):
                        df.iloc[j,i]=0
            #print('\n')
    return df

In [11]:
df1=fillN(df1)

In [12]:
df1=label(df1)

In [15]:
X = df1.iloc[0:,1:53]
y= df1.iloc[0:,56]

In [16]:
def createNewY(y_test):
    newy=[]
    # 0, 10, 100, 1000, 10000, 100000, 1000000, 10000000
    for i in y_test:
        if(i<10):
            newy.append([1,0,0,0,0,0,0,0])
        elif(i>=10 and i<100):
            newy.append([0,1,0,0,0,0,0,0])
        elif(i>=100 and i<1000):
            newy.append([0,0,1,0,0,0,0,0])
        elif(i>=1000 and i<10000):
            newy.append([0,0,0,1,0,0,0,0])
        elif(i>=10000 and i<100000):
            newy.append([0,0,0,0,1,0,0,0])
        elif(i>=100000 and i<1000000):
            newy.append([0,0,0,0,0,1,0,0])
        elif(i>=1000000 and i<10000000):
            newy.append([0,0,0,0,0,0,1,0])
        elif(i>=10000000):
            newy.append([0,0,0,0,0,0,0,1])
    return newy
def createNewY1(y_test):
    newy=[]
    # 0, 10, 100, 1000, 10000, 100000, 1000000, 10000000
    for i in y_test:
        if(i<10):
            newy.append(0)
        elif(i>=10 and i<100):
            newy.append(1)
        elif(i>=100 and i<1000):
            newy.append(2)
        elif(i>=1000 and i<10000):
            newy.append(3)
        elif(i>=10000 and i<100000):
            newy.append(4)
        elif(i>=100000 and i<1000000):
            newy.append(5)
        elif(i>=1000000 and i<10000000):
            newy.append(6)
        elif(i>=10000000):
            newy.append(7)
    return newy

# with Yorkpaid

In [17]:
newy=createNewY(y)
newy=np.asarray(newy)
X_train, X_test, y_train, y_test = train_test_split(X, newy, test_size = 0.2)
y_train[0]

array([0, 1, 0, 0, 0, 0, 0, 0])

In [18]:
classifier = Sequential()
# Adding the input layer and the first hidden layer
classifier.add(Dense(activation="relu", input_dim=X_train.shape[1], units=6, kernel_initializer="uniform"))
# Adding the second hidden layer
classifier.add(Dropout(0.2))
classifier.add(Dense(activation="relu", units=10, kernel_initializer="uniform"))
# Adding the output layer
classifier.add(Dropout(0.2))
classifier.add(Dense(activation="relu", units=8, kernel_initializer="uniform"))
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
classifier.fit(X_train, y_train, batch_size = 100, epochs = 30)

Epoch 1/30
16075/16075 [==============================] - 1s 39us/step - loss: 1.3343 - acc: 0.8362
Epoch 2/30
16075/16075 [==============================] - 0s 14us/step - loss: 0.2570 - acc: 0.8755
Epoch 3/30
16075/16075 [==============================] - 0s 15us/step - loss: 0.2041 - acc: 0.8778
Epoch 4/30
16075/16075 [==============================] - 0s 12us/step - loss: 0.1870 - acc: 0.9160
Epoch 5/30
16075/16075 [==============================] - 0s 12us/step - loss: 0.1813 - acc: 0.9261
Epoch 6/30
16075/16075 [==============================] - 0s 12us/step - loss: 0.1804 - acc: 0.9268
Epoch 7/30
16075/16075 [==============================] - 0s 11us/step - loss: 0.1802 - acc: 0.9271
Epoch 8/30
16075/16075 [==============================] - 0s 11us/step - loss: 0.1799 - acc: 0.9273
Epoch 9/30
16075/16075 [==============================] - 0s 12us/step - loss: 0.1799 - acc: 0.9274
Epoch 10/30
16075/16075 [==============================] - 0s 12us/step - loss: 0.1798 - acc: 0.9274

In [19]:
y_pred=classifier.predict(X_test)
y_pred
score = classifier.evaluate(X_test, y_test, batch_size=2000)
score

4019/4019 [==============================] - 0s 11us/step


[0.18152162001467545, 0.9247325090485208]

In [22]:
def accuracy2(y_pred,y_test):
    l=len(y_pred)
    count=0
    for i in range(l):
        m=max(y_pred[i])
       # print(m)
        index= np.where(y_pred[i]==m)
        #print(index)
        one = np.where(y_test[i]==1)
        #print(one)
        if(one[0]==index[0]):
            count+=1
       # elif(one[0]==(index[0]-1) or one[0]==(index[0]+1)):
        #    count+=0.5
    return count/l*100

In [23]:
accuracy2(y_pred,y_test)

69.89300821099776

# with Incurred amount

In [30]:
y= df1.iloc[0:,59]
newy=createNewY(y)
newy=np.asarray(newy)
X_train, X_test, y_train, y_test = train_test_split(X, newy, test_size = 0.2)
y_train[0]
classifier = Sequential()
# Adding the input layer and the first hidden layer
classifier.add(Dense(activation="relu", input_dim=X_train.shape[1], units=6, kernel_initializer="uniform"))
# Adding the second hidden layer
classifier.add(Dropout(0.2))
classifier.add(Dense(activation="relu", units=10, kernel_initializer="uniform"))
# Adding the output layer
classifier.add(Dropout(0.2))
classifier.add(Dense(activation="relu", units=8, kernel_initializer="uniform"))
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
classifier.fit(X_train, y_train, batch_size = 100, epochs = 30)
y_pred=classifier.predict(X_test)

Epoch 1/30
16075/16075 [==============================] - 1s 35us/step - loss: 0.8087 - acc: 0.8572
Epoch 2/30
16075/16075 [==============================] - 0s 13us/step - loss: 0.3309 - acc: 0.8715
Epoch 3/30
16075/16075 [==============================] - ETA: 0s - loss: 0.2987 - acc: 0.874 - 0s 13us/step - loss: 0.2965 - acc: 0.8746
Epoch 4/30
16075/16075 [==============================] - 0s 12us/step - loss: 0.2874 - acc: 0.8755
Epoch 5/30
16075/16075 [==============================] - 0s 12us/step - loss: 0.2866 - acc: 0.8766
Epoch 6/30
16075/16075 [==============================] - 0s 12us/step - loss: 0.2862 - acc: 0.8755
Epoch 7/30
16075/16075 [==============================] - 0s 14us/step - loss: 0.2859 - acc: 0.8762
Epoch 8/30
16075/16075 [==============================] - 0s 16us/step - loss: 0.2858 - acc: 0.8749
Epoch 9/30
16075/16075 [==============================] - 0s 13us/step - loss: 0.2857 - acc: 0.8753
Epoch 10/30
16075/16075 [==============================] - 0s 

In [31]:
score = classifier.evaluate(X_test, y_test, batch_size=2000)
score

4019/4019 [==============================] - 0s 24us/step


[0.28647895469894513, 0.8750932933412757]